**<font color=blue size=5 face=微軟雅黑>刪除特定副檔名的檔案</font>**

In [1]:
import os
import sys
import csv
import glob
import time
import string
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

**<font color=green size=3 face=微軟雅黑>查詢資料夾下的所有檔案名稱</font>**
https://medium.com/ai%E5%8F%8D%E6%96%97%E5%9F%8E/python-%E6%9F%A5%E8%A9%A2%E8%B3%87%E6%96%99%E5%A4%BE%E4%B8%8B%E7%9A%84%E6%89%80%E6%9C%89%E6%AA%94%E6%A1%88%E5%90%8D%E7%A8%B1-45c6ec75c430

In [59]:
# 列出資料夾底下所有檔案名稱
dirPath = r"Midi/test"
num = [f for f in os.listdir(dirPath) if os.path.isfile(os.path.join(dirPath,f))]
print('File count in Folder ' + dirPath + ' : ' + str(len(num)))
print(num)

File count in Folder Midi/test : 279
['10-02-i.mid', '1st_Hungarian_DancePNO.mid', '8n_beethoven_sonata_1.pdf', '8n_beethoven_sonata_10.pdf', '8n_beethoven_sonata_11.pdf', '8n_beethoven_sonata_12.pdf', '8n_beethoven_sonata_13.pdf', '8n_beethoven_sonata_14.pdf', '8n_beethoven_sonata_15.pdf', '8n_beethoven_sonata_16.pdf', '8n_beethoven_sonata_17.pdf', '8n_beethoven_sonata_18.pdf', '8n_beethoven_sonata_19.pdf', '8n_beethoven_sonata_2.pdf', '8n_beethoven_sonata_21.pdf', '8n_beethoven_sonata_22.pdf', '8n_beethoven_sonata_23.pdf', '8n_beethoven_sonata_24.pdf', '8n_beethoven_sonata_25.pdf', '8n_beethoven_sonata_26.pdf', '8n_beethoven_sonata_27.pdf', '8n_beethoven_sonata_28.pdf', '8n_beethoven_sonata_29.pdf', '8n_beethoven_sonata_3.pdf', '8n_beethoven_sonata_30.pdf', '8n_beethoven_sonata_31.pdf', '8n_beethoven_sonata_32.pdf', '8n_beethoven_sonata_4.pdf', '8n_beethoven_sonata_5.pdf', '8n_beethoven_sonata_6.pdf', '8n_beethoven_sonata_7.pdf', '8n_beethoven_sonata_8.pdf', '8n_beethoven_sonata_9.pd

In [60]:
# 將檔案名稱分割成檔名和副檔名，分別放進dataframe
list_filename = []
list_extension = []
df = pd.DataFrame()

for i in range(len(num)):
    base = 'Midi/test/'
    path = base + num[i]
    # 分離文件名稱和副檔名
    a, b = os.path.split(path)
    # 利用"."分割
    blist = b.rsplit('.',1)
    list_filename.append(blist[0])
    list_extension.append(blist[1])

df['Filename'] = pd.Series(list_filename)
df['Extension'] = pd.Series(list_extension)
df.to_csv('csv/filename_split.csv', index = False)
df

,Filename,Extension
0,10-02-i,mid
1,1st_Hungarian_DancePNO,mid
2,8n_beethoven_sonata_1,pdf
3,8n_beethoven_sonata_10,pdf
4,8n_beethoven_sonata_11,pdf
...,...,...
274,Vexations_PN,mid
275,wall_street_rag,mid
276,Waltz_Opus_70_No._1PNO,mid
277,weeping_willow_PNO,mid


In [61]:
# 列出副檔名不是「.mid」的
df = df[df.Extension != 'mid']
# index重新排列
df.index = range(len(df))
df

,Filename,Extension
0,8n_beethoven_sonata_1,pdf
1,8n_beethoven_sonata_10,pdf
2,8n_beethoven_sonata_11,pdf
3,8n_beethoven_sonata_12,pdf
4,8n_beethoven_sonata_13,pdf
...,...,...
105,sp31,pdf
106,sp32,pdf
107,sp33,pdf
108,sp34,pdf


In [62]:
for j in range(len(df)):
    filename = df['Filename'][j]
    extension = df['Extension'][j]
    delete_path = 'Midi/test/' + filename + '.' + extension
    #print(delete_path)
    # 刪除副檔名為「.pdf」的檔案
    os.remove(delete_path)
print('Extension of [ .' + extension + ' ] deleted !')

Extension of [ .pdf ] deleted !


In [63]:
# 統計資料夾檔案個數
DIR = 'Midi/test'
file_count = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
print(str(file_count) + ' files in [ ' + DIR + ' ]')

169 files in [ Midi/test ]


**<font color=blue size=5 face=微軟雅黑>8notes網站抓不同調性的MIDI檔案</font>**
https://www.8notes.com/key_signatures/default.asp?akey=a&instrument_ID=1

In [24]:
# 爬取MIDI類別
base_url = 'https://www.8notes.com/key_signatures/default.asp?akey=a&instrument_ID=1'
page = requests.get(base_url)
soup = BeautifulSoup(page.text, "html.parser")

# 選擇 class為 'alphabet2'
results = soup.findAll("a", {"class": "alphabet2"})
#print(results)

list_key = []
list_href = []
df_midikey = pd.DataFrame()

for r in results:
    list_key.append(r.text)
    list_href.append('https://www.8notes.com/key_signatures/' + r['href'])
    #print(r.text, r['href'])

df_midikey['Key'] = pd.Series(list_key)
df_midikey['Url'] = pd.Series(list_href)

# 移除非key調性的row
df_midikey = df_midikey[df_midikey.Key != 'Change Instrument']
df_midikey = df_midikey[df_midikey.Key != 'Show all instruments']
df_midikey.to_csv(r'csv/df_midikey.csv',index=False)
df_midikey

,Key,Url
0,A,https://www.8notes.com/key_signatures/default....
1,B,https://www.8notes.com/key_signatures/default....
2,C,https://www.8notes.com/key_signatures/default....
3,D,https://www.8notes.com/key_signatures/default....
4,E,https://www.8notes.com/key_signatures/default....
5,F,https://www.8notes.com/key_signatures/default....
6,G,https://www.8notes.com/key_signatures/default....
7,Ab,https://www.8notes.com/key_signatures/default....
8,Bb,https://www.8notes.com/key_signatures/default....
9,Db,https://www.8notes.com/key_signatures/default....


In [25]:
def get_nextpage(url):

    page1 = requests.get(url)
    soup1 = BeautifulSoup(page1.text, "html.parser")
    btn = soup1.find('div', class_='pagination')
    #btn_next_li = btn.find_all('li')
    btn_next = btn.find_all('a', class_='prevnext')

    # 目前頁面網址
    page_href = btn_next[1]['href']

    # 下一頁網址
    next_page_url = 'https://www.8notes.com' + page_href
    url = next_page_url

    try:
        if next_page_url == 'https://www.8notes.comjavascript:;':
            pass
        else:
            list_next_page.append(url)
            get_nextpage(url=next_page_url)
    except:
        pass
    
    return list_next_page

In [26]:
def get_title(df_midikey):
    
    # 迴圈抓取下一頁(利用list_next_page陣列的每一頁網址抓取)
    for i in range(len(list_next_page)):
        v = list_next_page[i]
        page2 = requests.get(v)
        soup2 = BeautifulSoup(page2.text, "html.parser")
        
        title = soup2.findAll("div", {"class": "redtitle"})
        #print(title)
        title_url = soup2.findAll("a", {"class": "listboxrow"})
        #print(title_url)

        for t, u in zip(title, title_url):
            list_title.append(t.text)

            # 比對字串：url如果有包含'/scores/'就保留，否則為空白
            if str.find(u['href'],'/scores/') != -1:
                list_title_url.append(u['href'])
            else:
                list_title_url.append('')
    
    # 曲目名稱和曲目頁面網址放入dataframe中
    df_midikey_url['Title'] = pd.Series(list_title)
    df_midikey_url['Url'] = pd.Series(list_title_url)
    
    return df_midikey_url

In [27]:
def get_eachpage_midi(df_midikey_url):

    for i in range(len(df_midikey_url)):

        # 把曲目頁面網址用完整並導向MIDI檔案格式
        x = df_midikey_url['Url'][i]

        # URL是否為空
        if x != '':
            midi_url = 'https://www.8notes.com' + x + '?ftype=midi'
            #print(midi_url)
            list_midi_url.append(midi_url)

            page3 = requests.get(midi_url)
            soup3 = BeautifulSoup(page3.text, "html.parser")

            # MIDI下載按鈕的連結
            results = soup3.find("a", {"class": "transp_alphabet"}).attrs["href"]
            #print(results)
            
            # 判斷midi下載網址是否能下載，若回傳的href為/school/midi，則不執行midi下載指令
            if results == '/school/midi/':
                download_url = 'https://www.8notes.com' + results
                list_download_url.append(download_url)
            else:
                download_url = 'https://www.8notes.com' + results
                list_download_url.append(download_url)
                #print('Download:', download_url)
                
                instrument = download_url.split('/')[::-1][1]
                filename = download_url.split('/')[::-1][0]
                #print(filename)
                
                if instrument == 'piano':
                    mid_file = requests.get(download_url, stream=True)

                    # MIDI開始下載
                    with open(folder_path + '/' + filename, 'wb') as saveMidFile:
                        saveMidFile.write(mid_file.content)
                        #print('Downloaded {} successfully.'.format(filename))
    
        # 如果URL為空，就append空白進去dataframe
        else:
            list_midi_url.append('')
            list_download_url.append('')
            pass
    
    # 曲目midi頁面和midi下載路徑新增放入dataframe中    
    df_midikey_url['MidiUrl'] = pd.Series(list_midi_url)
    df_midikey_url['DownloadUrl'] = pd.Series(list_download_url)
    
    #return list_midi_url, list_download_url, df_midikey_url
    return df_midikey_url

In [63]:
df_midikey = pd.read_csv(r'csv/df_midikey.csv')
list_next_page = []
list_title = []
list_title_url = []
list_midi_url = []
list_download_url = []
df_midikey_url = pd.DataFrame()

for count in range(19, 20):
    print('Round:', count)
    
    key = df_midikey['Key'][count]
    url = df_midikey['Url'][count]
    folder_path = 'Midi/KeySignatures/' + str(count).zfill(2) + '_' + str(key)
    os.mkdir(folder_path)
    print('Create Folder [ ' + folder_path + ' ] Finish!')
    
    page_check = requests.get(url)
    soup_check = BeautifulSoup(page_check.text, "html.parser")
    btn_check = soup_check.find('div', class_='pagination')
    
    # 只有一頁，沒有next_page
    if btn_check == None:
        list_next_page = [url]
    # 兩頁以上，有next_page
    else:
        list_next_page = get_nextpage(url)
        list_next_page.insert(0, url)
        #print(list_next_page)
    
    df_midikey_url = get_title(df_midikey)
    a = 'csv/KeySignatures/' + str(count).zfill(2) + '_' + str(key) + '_df_midikey_url.csv'
    print('Saved in ' + str(a))
    df_midikey_url.to_csv(a, index=False)

    get_eachpage_midi(df_midikey_url)
    b = 'csv/KeySignatures/' + str(count).zfill(2) + '_' + str(key) + '_df_midikey_url_final.csv'
    print('Saved in ' + str(b))
    df_midikey_url.to_csv(b, index=False)

    DIR = folder_path  #要統計的資料夾
    file_count = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
    print(str(file_count) + ' files in [ ' + folder_path + ' ]')
    print('-------------------------------------------------------------------')

Round: 19
Create Folder [ Midi/KeySignatures/19_Gm ] Finish!
['https://www.8notes.com/key_signatures/default.asp?akey=gm&instrument_ID=1', 'https://www.8notes.com/key_signatures/?instrument_ID=1&akey=gm&pageA=2', 'https://www.8notes.com/key_signatures/?instrument_ID=1&akey=gm&pageA=3']
Saved in csv/KeySignatures/19_Gm_df_midikey_url.csv
Saved in csv/KeySignatures/19_Gm_df_midikey_url_final.csv
49 files in [ Midi/KeySignatures/19_Gm ]
-------------------------------------------------------------------


In [111]:
# 取得樂器類別，是piano的才下載
download_url = 'https://www.8notes.com/school/midi/violin/gardel_por.mid'
instrument = download_url.split('/')[::-1][1]
instrument

'violin'

In [61]:
# 確認是否有下一頁的按鈕(是否兩頁以上)
pageT = requests.get('https://www.8notes.com/key_signatures/default.asp?akey=db&instrument_ID=1')
soupT = BeautifulSoup(pageT.text, "html.parser")
btnT = soupT.find('div', class_='pagination')
#btn_nextT = btnT.find_all('a', class_='prevnext')
if btnT == None:
    print('Empty!')

Empty!
